## imports e defs

In [28]:
from SPARQLWrapper import SPARQLWrapper, JSON
import time
import pandas as pd
import datetime

In [3]:
ENDPOINT = "http://10.33.96.18:7200/repositories/GRAFO_SEFAZMA_PRODUCAO"
sparql = SPARQLWrapper(ENDPOINT)

In [29]:
def execute(query,sparql=sparql):
    start_time = time.time()
    try:
        print(f"Executando consulta em {datetime.datetime.now()}:\n\n{query}\n\n-------------------------------------------------")
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        end_time = time.time()
    except:
        results = None
    timeExecution = end_time - start_time
    return timeExecution, results

In [5]:
def size(r):
    if r == None:
        return 0
    return len(r['results']['bindings'])

## Tempo para pegar triplas genêricas?

### código

In [30]:
query= """
select * where { 
	?s ?p ?o .
} limit 100 
"""
t, r = execute(query)

Executando consulta em 2022-05-24 12:26:47.967089:


select * where { 
	?s ?p ?o .
} limit 100 


-------------------------------------------------


### resultado

In [25]:
pd_0 = pd.DataFrame([{'tempo':t,'triplas':size(r)}])
pd_0.to_csv("pd0.csv",index=False)
print("pd0.csv salvo com sucesso!!!")
pd_0


pd0.csv salvo com sucesso!!!


,tempo,triplas
0,1.209397,100


## Quantidade de classes:

### código

In [8]:
ENDPOINT_ONTOLGY = "http://10.33.96.18:7200/repositories/ONTOLOGIA_DOMINIO"
sparql_ontology = SPARQLWrapper(ENDPOINT_ONTOLGY)

In [9]:
query= """
		prefix owl: <http://www.w3.org/2002/07/owl#>
        prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        select ?class ?label where { 
            {
                ?class a owl:Class.
            }
            UNION{
                ?class a rdfs:Class.
            }
            OPTIONAL{?class rdfs:label ?l}
            BIND(COALESCE(?l,?class) AS ?label)
            FILTER(!CONTAINS(STR(?class),"http://www.w3.org/2000/01/rdf-schema#"))
            FILTER(!CONTAINS(STR(?class),"http://www.w3.org/2001/XMLSchema#"))
            FILTER(!CONTAINS(STR(?class),"http://www.w3.org/1999/02/22-rdf-syntax-ns#"))
            FILTER(!CONTAINS(STR(?class),"http://www.w3.org/2002/07/owl#"))
        } ORDER BY ?label  
"""
print("Consultando classes...")
t, r = execute(query,sparql=sparql_ontology)

Consultando classes...


In [10]:
classes = []
for result in r['results']['bindings']:
    classes.append({'label': result['label']['value'],'uri':result['class']['value']})

### Resultado

In [11]:
print(len(classes))

150


## Tempo para recursos de cada classe

### código

In [16]:
results = []
recurso_classe = []
for classe in classes:
    query= f"""
            prefix owl: <http://www.w3.org/2002/07/owl#>
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            select * where {{ 
                ?s a <{classe['uri']}>;
                    ?p ?o.
            }} LIMIT 100
    """
    t, r = execute(query)
    if size(r) > 0:
        recurso_classe.append([classe['label'],r['results']['bindings'][0]['s']['value']])
    results.append({'tempo':t,'classe':classe['label'],'triplas':size(r)})
    
pd_1 = pd.DataFrame(results)
pd_1.to_csv("pd1.csv",index=False)
print("pd1.csv salvo com sucesso!!!")

Executando consulta:


            prefix owl: <http://www.w3.org/2002/07/owl#>
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            select * where { 
                ?s a <http://www.sefaz.ma.gov.br/ontology/Acordao>;
                    ?p ?o.
            } LIMIT 100
    

-------------------------------------------------
pd1.csv salvo com sucesso!!!


### resultado

In [ ]:
pd_1

,tempo,classe,triplas
0,3.437244,Acórdão,100


## Recurso específico por classe:

### código

In [17]:
results = []
for recurso in recurso_classe:
    query= f"""
            prefix owl: <http://www.w3.org/2002/07/owl#>
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            select * where {{ 
                <{recurso[1]}> ?p ?o.
            }} LIMIT 100
    """
    t, r = execute(query)
    results.append({'tempo':t,'classe':recurso[0],'triplas':size(r)})
    
pd_2 = pd.DataFrame(results)
pd_2.to_csv("pd2.csv",index=False)
print("pd2.csv salvo com sucesso!!!")

Executando consulta:


            prefix owl: <http://www.w3.org/2002/07/owl#>
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            select * where { 
                <http://www.sefaz.ma.gov.br/resource/Restricoes/Acordao/14769662491-10_05_2017-00371520167> ?p ?o.
            } LIMIT 100
    

-------------------------------------------------
pd2.csv salvo com sucesso!!!


### Resultado

In [ ]:
pd_2

,tempo,classe,triplas
0,0.282697,Acórdão,5


## Label para recurso:

### código

In [18]:
results = []
for recurso in recurso_classe:
    query= f"""
            prefix owl: <http://www.w3.org/2002/07/owl#>
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            select * where {{ 
                <{recurso[1]}> rdfs:label ?o.
            }} LIMIT 100
    """
    
    t, r = execute(query)
    results.append({'tempo':t,'classe':recurso[0],'triplas':size(r)})
    
pd_3 = pd.DataFrame(results)
pd_3.to_csv("pd3.csv",index=False)
print("pd3.csv salvo com sucesso!!!")

Executando consulta:


            prefix owl: <http://www.w3.org/2002/07/owl#>
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            select * where { 
                <http://www.sefaz.ma.gov.br/resource/Restricoes/Acordao/14769662491-10_05_2017-00371520167> rdfs:label ?o.
            } LIMIT 100
    

-------------------------------------------------
pd3.csv salvo com sucesso!!!


### resultado

In [ ]:
pd_3

,tempo,classe,triplas
0,0.162196,Acórdão,1


## Tempo para cada propriedade por classe

### código

In [19]:
propriedades = {}
results = []
for classe in classes:
    query= f"""
            prefix owl: <http://www.w3.org/2002/07/owl#>
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            select DISTINCT ?p ?l where {{ 
                {{
                    ?p rdfs:domain <{classe['uri']}>;
                    rdfs:label ?l.
                }}
                UNION{{
                    <{classe['uri']}> rdfs:subClassOf [ rdf:type owl:Restriction ; owl:onProperty ?p; owl:someValuesFrom ?r].
                    ?p rdfs:label ?l.
                }}
                UNION{{
                    <{classe['uri']}> rdfs:subClassOf [ rdf:type owl:Restriction ; owl:onProperty ?p; owl:allValuesFrom ?r].
                    ?p rdfs:label ?l.
                }}
            }}
    """
    t, r = execute(query,sparql=sparql_ontology)
    propriedades[classe['label']] = []
    for result in r['results']['bindings']:
        propriedades[classe['label']].append([result['l']['value'],result['p']['value']])
        p_url = result['p']['value']
        query= f"""
            prefix owl: <http://www.w3.org/2002/07/owl#>
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            select * where {{ 
                ?s a <{classe['uri']}>;
                    <{p_url}> ?o.
            }} LIMIT 100
        """
        
        t, r = execute(query)
        results.append({'tempo':t,'classe':classe['label'],'propriedade':result['l']['value'],'triplas':size(r)})
    
pd_4 = pd.DataFrame(results)
pd_4.to_csv("pd4.csv",index=False)
print("pd4.csv salvo com sucesso!!!")

Executando consulta:


            prefix owl: <http://www.w3.org/2002/07/owl#>
            prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            select * where { 
                ?s a <http://www.sefaz.ma.gov.br/ontology/Acordao>;
                    <http://www.sefaz.ma.gov.br/ontology/data_final> ?o.
            } LIMIT 100
        

-------------------------------------------------
pd4.csv salvo com sucesso!!!


### resultado

In [ ]:
pd_4

,tempo,classe,propriedade,triplas
0,2.574672,Acórdão,data final,0


## Relações de entrada e saída por recurso

### código

In [20]:
results = []
for recurso in recurso_classe:
    query= f"""
            SELECT * where {{ 
                BIND(<{recurso[1]}> as ?node)
                {{
                    ?node ?p ?o .    
                    filter(?p != <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>)
                    filter(isURI(?o))
                }}
                OPTIONAL{{
                    ?s ?p2 ?node.
                    filter(?p2 != <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>)
                    filter(isURI(?s))
                }}
            }}
    """
    
    t, r = execute(query)
    results.append({'tempo':t,'classe':recurso[0],'recurso':recurso[1],'triplas':size(r)})
    
pd_5 = pd.DataFrame(results)
pd_5.to_csv("pd5.csv",index=False)
print("pd5.csv salvo com sucesso!!!")

Executando consulta:


            SELECT * where { 
                BIND(<http://www.sefaz.ma.gov.br/resource/Restricoes/Acordao/14769662491-10_05_2017-00371520167> as ?node)
                {
                    ?node ?p ?o .    
                    filter(?p != <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>)
                    filter(isURI(?o))
                }
                OPTIONAL{
                    ?s ?p2 ?node.
                    filter(?p2 != <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>)
                    filter(isURI(?s))
                }
            }
    

-------------------------------------------------
pd5.csv salvo com sucesso!!!


### resultado

In [ ]:
pd_5

,tempo,classe,recurso,triplas
0,2.417738,Acórdão,http://www.sefaz.ma.gov.br/resource/Restricoes...,1


## Cadeia de 2 relações por recurso

### código

In [21]:
results = []
for recurso in recurso_classe:
    query= f"""
            SELECT * where {{ 
                BIND(<{recurso[1]}> as ?node)  
                ?node ?p ?o .    
                filter(?p != <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>)
                filter(isURI(?o))
                ?o ?p3 ?o2.
                filter(?p3 != <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>)
                filter(isURI(?o2))
            }}
    """
    
    t, r = execute(query)
    results.append({'tempo':t,'classe':recurso[0],'recurso':recurso[1],'triplas':size(r)})
pd_6 = pd.DataFrame(results)
pd_6.to_csv("pd6.csv",index=False)
print("pd6.csv salvo com sucesso!!!")

Executando consulta:


            SELECT * where { 
                BIND(<http://www.sefaz.ma.gov.br/resource/Restricoes/Acordao/14769662491-10_05_2017-00371520167> as ?node)  
                ?node ?p ?o .    
                filter(?p != <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>)
                filter(isURI(?o))
                ?o ?p3 ?o2.
                filter(?p3 != <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>)
                filter(isURI(?o2))
            }
    

-------------------------------------------------
pd6.csv salvo com sucesso!!!


### resultado

In [ ]:
pd_6

,tempo,classe,recurso,triplas
0,1.1797,Acórdão,http://www.sefaz.ma.gov.br/resource/Restricoes...,1


## Pega atributos de recursos

## código

In [22]:
results = []
for recurso in recurso_classe:
    query= f"""

            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?node ?property ?value WHERE{{
                BIND(<{recurso[1]}> as ?node)  
                ?node ?property_ ?value
                FILTER(!isURI(?value))
                BIND((REPLACE(STR(?property_),".*(/|#)","")) as ?property)
            }}
    """
    
    t, r = execute(query)
    results.append({'tempo':t,'classe':recurso[0],'recurso':recurso[1],'triplas':size(r)})
pd_7 = pd.DataFrame(results)
pd_7.to_csv("pd7.csv",index=False)
print("pd7.csv salvo com sucesso!!!")

Executando consulta:



            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?node ?property ?value WHERE{
                BIND(<http://www.sefaz.ma.gov.br/resource/Restricoes/Acordao/14769662491-10_05_2017-00371520167> as ?node)  
                ?node ?property_ ?value
                FILTER(!isURI(?value))
                BIND((REPLACE(STR(?property_),".*(/|#)","")) as ?property)
            }
    

-------------------------------------------------
pd7.csv salvo com sucesso!!!


### resultado

In [ ]:
pd_7

,tempo,classe,recurso,triplas
0,0.773208,Acórdão,http://www.sefaz.ma.gov.br/resource/Restricoes...,3
